In [74]:
import pandas as pd
from backend.data_center.kline_data.kline_data_collector import *
tv = KlineDataCollector()
file_abspath = tv.get_abspath(symbol='SOL', interval=Interval.in_daily)
df = pd.read_csv(f"{file_abspath}")
df

error while signin
you are using nologin method, data you access may be limited


,datetime,symbol,open,high,low,close,volume
0,2020-08-11 08:00:00,Binance:SOLUSDT,2.8500,3.5208,2.8433,3.2985,1552384.780
1,2020-08-12 08:00:00,Binance:SOLUSDT,3.2985,3.9289,3.0800,3.7558,1737042.950
2,2020-08-13 08:00:00,Binance:SOLUSDT,3.7500,4.1387,3.5003,3.7300,1685759.240
3,2020-08-14 08:00:00,Binance:SOLUSDT,3.7207,3.7676,3.3210,3.4099,1474161.790
4,2020-08-15 08:00:00,Binance:SOLUSDT,3.4181,3.7400,3.1500,3.1730,1070233.200
...,...,...,...,...,...,...,...
1514,2024-10-03 08:00:00,Binance:SOLUSDT,140.1800,142.9600,133.1100,136.8200,3760168.377
1515,2024-10-04 08:00:00,Binance:SOLUSDT,136.8100,144.5500,135.9500,143.2400,3137296.636
1516,2024-10-05 08:00:00,Binance:SOLUSDT,143.2400,144.9300,141.0600,142.5900,1483444.653
1517,2024-10-06 08:00:00,Binance:SOLUSDT,142.5900,146.7500,142.0000,146.4800,1415860.027


In [75]:
df['prev_low_1'] = df['low'].shift(1)
df['prev_high_1'] = df['high'].shift(1)
df['next_close_3'] = df['close'].shift(-3)
df['next_close_2'] = df['close'].shift(-2)
df['next_close_1'] = df['close'].shift(-1)
df['next_close_5'] = df['close'].shift(-5)
df['next_open_2'] = df['open'].shift(-2)
df['next_open_1'] = df['open'].shift(-1)
df.dropna(inplace=True)
# calculate the close price in a quarter at the end of the day
df['attack_buy_pattern'] = df['close'] < df['prev_low_1'] 
df['prev_is_abp'] = df['attack_buy_pattern'].shift(1)
df.dropna(inplace=True)


In [76]:
df['buy_sig'] = (df['prev_is_abp'] == True) & (df['close']>df['prev_high_1'])
df[df['buy_sig'] == True]


,datetime,symbol,open,high,low,close,volume,prev_low_1,prev_high_1,next_close_3,next_close_2,next_close_1,next_close_5,next_open_2,next_open_1,attack_buy_pattern,prev_is_abp,buy_sig
55,2020-10-05 08:00:00,Binance:SOLUSDT,2.5181,2.6830,2.4561,2.6522,9.125872e+05,2.4200,2.5898,2.3638,2.3005,2.1740,2.3999,2.1721,2.6486,False,True,True
139,2020-12-28 08:00:00,Binance:SOLUSDT,1.3041,1.5223,1.2867,1.5172,2.158621e+06,1.2616,1.4170,1.5073,1.5350,1.6704,1.7999,1.6734,1.5187,False,True,True
143,2021-01-01 08:00:00,Binance:SOLUSDT,1.5088,1.8600,1.4990,1.8421,4.421991e+06,1.4200,1.5523,2.4909,2.1779,1.7999,1.9414,1.7998,1.8421,False,True,True
149,2021-01-07 08:00:00,Binance:SOLUSDT,1.9404,2.6500,1.9214,2.3635,3.086101e+07,1.9017,2.1890,3.4490,3.3829,3.2214,3.6354,3.2120,2.3655,False,True,True
154,2021-01-12 08:00:00,Binance:SOLUSDT,3.1016,3.8800,3.0580,3.6354,7.075912e+06,2.5795,3.5282,3.2475,3.2921,3.5242,3.8268,3.5243,3.6354,False,True,True
227,2021-03-26 08:00:00,Binance:SOLUSDT,12.9789,14.7921,12.3043,14.7664,2.972282e+06,12.4700,13.7329,18.9504,18.0645,16.4589,19.3861,16.4516,14.7751,False,True,True
250,2021-04-18 08:00:00,Binance:SOLUSDT,24.8517,32.9900,21.1000,32.3358,1.169940e+07,24.8181,26.7914,32.5534,31.7028,31.4711,37.3893,31.5303,32.3324,False,True,True
337,2021-07-14 08:00:00,Binance:SOLUSDT,29.1540,31.5000,27.3630,31.3650,3.434185e+06,28.8100,31.0960,26.8020,26.2710,28.5450,24.5090,28.5620,31.3650,False,True,True
344,2021-07-21 08:00:00,Binance:SOLUSDT,23.4840,27.4610,22.8000,26.7000,4.561993e+06,22.1000,24.9820,28.6160,28.5160,27.6990,28.1840,27.7000,26.7060,False,True,True
407,2021-09-22 08:00:00,Binance:SOLUSDT,124.3000,151.3600,122.1000,148.0200,7.460596e+06,116.0000,144.6400,136.0600,139.1300,149.9100,136.3500,149.9000,148.0200,False,True,True


In [68]:
# to check
df['take_profit'] = ((df['buy_sig'] == True) & (df['next_close_2'] > df['next_open_1']))
                     # & (df['high'] > df['prev_high_1']))
df[df['take_profit'] == True]

,datetime,symbol,open,high,low,close,volume,prev_low_1,prev_high_1,next_close_3,next_close_2,next_close_1,next_close_5,attack_buy_pattern,prev_is_abp,buy_sig,next_open_2,next_open_1,take_profit
179,2018-02-12 08:00:00,Binance:BTCUSDT,8063.82,8989.00,8053.00,8903.00,41987.850490,7726.53,8549.00,10000.09,9449.99,8539.90,11039.55,False,True,True,8535.17,8903.00,True
285,2018-05-29 08:00:00,Binance:BTCUSDT,7099.00,7540.00,7032.95,7461.29,39406.839520,7058.02,7437.00,7521.01,7485.01,7375.96,7714.26,False,True,True,7382.07,7467.98,True
316,2018-06-29 08:00:00,Binance:BTCUSDT,5851.01,6300.00,5780.00,6197.92,33911.287278,5827.00,6173.01,6615.29,6356.81,6390.07,6586.98,False,True,True,6391.08,6197.92,True
444,2018-11-04 08:00:00,Binance:BTCUSDT,6388.00,6525.00,6359.00,6485.85,10592.394943,6345.00,6439.97,6578.46,6519.11,6468.99,6419.99,False,True,True,6468.99,6485.85,True
565,2019-03-05 08:00:00,Binance:BTCUSDT,3716.10,3877.10,3703.55,3857.73,32962.536162,3670.69,3830.00,3864.89,3873.64,3861.84,3916.82,False,True,True,3861.84,3857.58,True
662,2019-06-10 08:00:00,Binance:BTCUSDT,7627.57,8020.00,7511.00,7982.75,36756.078468,7506.66,7935.00,8218.54,8127.64,7884.90,8808.70,False,True,True,7884.90,7981.00,True
781,2019-10-07 08:00:00,Binance:BTCUSDT,7855.30,8299.92,7762.00,8190.09,52202.072297,7785.00,8153.87,8558.03,8560.74,8168.39,8300.09,False,True,True,8170.79,8190.82,True
869,2020-01-03 08:00:00,Binance:BTCUSDT,6965.49,7405.00,6871.04,7344.96,68428.500451,6924.74,7212.50,7758.00,7358.75,7354.11,8055.98,False,True,True,7354.19,7345.00,True
902,2020-02-05 08:00:00,Binance:BTCUSDT,9197.02,9744.45,9177.22,9612.04,64870.415615,9093.01,9350.00,9895.05,9813.73,9772.00,9851.83,False,True,True,9772.00,9612.03,True
908,2020-02-11 08:00:00,Binance:BTCUSDT,9851.74,10323.59,9700.00,10223.08,62422.395224,9756.00,10188.00,10344.36,10229.63,10326.46,9917.27,False,True,True,10325.33,10223.08,True


In [69]:
len(df[df['take_profit'] == True])/len(df[df['buy_sig'] == True])

0.5555555555555556

In [52]:
df['profit_rate'] = (df['next_close_2'] - df['prev_high_1']) / df['prev_high_1']
df[df['take_profit'] == True]['profit_rate'].mean()

0.040313456276434484

In [53]:
df[df['prev_is_abp']==True]['profit_rate'].mean()

-0.04944443309516538